In [30]:
import os
import shutil
from glob import glob
import cv2
import keyboard
from IPython import display
import ultralytics
from ultralytics import YOLO


def save_traces(path_img_directory, path_traces_directory, probability_treshold = 0.5): 
    
    ''' 
    This function saves the images which contain traces in a new directory.
    The labels directory (.txt files) must be in the images directory and images must be saved as follows: image1.jpg, image2.jpg, image3.jpg, etc. The function uses "os", "shutil" and "glob" libraries to work.
    :param path_img_directory: path to the directory where the images are stored (after Yolo algorithm had run).
    :param path_traces_directory: path to the directory where the images with traces will be stored (if it doesn't exist, it will be created).
    :param probability_treshold: probability treshold to select a trace, set to 0.5 by default.
    :return: nothing
    '''
    
    imgnames = glob(fr'{path_img_directory}\*.jpg')
    probs = {}

    for imgname in imgnames:
        index = int(imgname[-5:-4])
        labelfile = os.path.join(rf"{path_img_directory}\labels", f"image{index}.txt")
        with open(labelfile, "r") as f:
            lines = f.readlines()
            outcomes = {}
            for line in lines:
                # read number and string in line
                number, string = line.split(" ", 1)
                outcomes[string] = float(number)
            probs[index] = outcomes
            
    if os.path.exists(path_traces_directory) == False:
        os.mkdir(path_traces_directory)

    for index, outcomes in probs.items():
        if outcomes['trace_frames\n'] > probability_treshold:
            shutil.copy2(rf'{path_img_directory}\image{index}.jpg', path_traces_directory)
    
    return


def record_video(delta_frame = 5):
    
    '''
    Record a video using a camera (we will use "Camo Studio" and a smartphone as a camera).
    Press "q" to stop recording.
    The function uses "cv2" and "keyboard" libraries to work.
    :params: delta_frame: number of frames to skip
    :return: frames: frames of the video
    '''
    
    cap = cv2.VideoCapture(1)
    i=0
    i_counter = 0
    frames = []
    
    while True:
        
        ret, frame = cap.read()
        
        if i_counter % delta_frame == 0: frames.append(frame)
        
        i_counter += 1
        
        if keyboard.is_pressed('q'):
            break
    
    cap.release()
    
    return frames

display.clear_output()
ultralytics.checks()

Ultralytics YOLOv8.2.76  Python-3.11.8 torch-2.4.0+cpu CPU (Intel Core(TM) i3-5010U 2.10GHz)
Setup complete  (4 CPUs, 11.9 GB RAM, 170.8/223.0 GB disk)


In [31]:
frames = record_video(delta_frame=5)

In [32]:
path_directory = rf'C:\Users\Black\PycharmProjects\MachineLearning\Cloud Chamber\YOLOv8\Yolo_Run'
nome_directory = rf'Run2'

model = YOLO("YOLO-CC-V1.pt")

results = model(frames, conf=0.25, task="classify", mode="predict", save=True, save_txt=True, save_conf=True, save_crop=False, project=path_directory, name=nome_directory)

0: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
1: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
2: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
3: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
4: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
5: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
6: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
7: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
8: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
9: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
10: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
11: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_frames 0.00, 153.3ms
12: 128x128 nothing_frames 1.00, doubt_frames 0.00, trace_fram

In [33]:
path_img = rf'{path_directory}\{nome_directory}'
path_traces = rf'{path_directory}\{nome_directory}\Traces_{nome_directory}'

save_traces(path_img, path_traces, 0.5)